<a href="https://colab.research.google.com/github/Bokdol11859/CNN-Pytorch/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter

In [ ]:
writer = SummaryWriter('runs/cifar10123')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#Hyper-parameter
num_epochs = 30
batch_size = 40
learning_rate = 0.001
num_classes = 10

In [ ]:
transform = transforms.Compose([transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])

train_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class VggNet(nn.Module):
  def __init__(self):
    super(VggNet, self).__init__()

    self.cnn_layers = Seqential(
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(True),

        nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(True),

        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(True),

        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(True),

        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(True),

        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(True),

        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(True),

        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(True),

        nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(True),

        nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(True),

        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(True),

        nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(True),

        nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
        nn.ReLU(True),

        nn.MaxPool2d(kernel_size=2, stride=2),
    )

    self.avgpool = nn.AdaptiveAvgPool2d(output_size = (7, 7))
    
    self.linear_layers = Sequential(
        nn.Linear(in_features=512*7*7, out_features=4096),
        nn.ReLU(True),
        nn.Dropout(p=0.5),
        
        nn.Linear(in_features=4096, out_features=4096),
        nn.ReLU(True),
        nn.Dropout(p=0.5),

        nn.Linear(in_features=4096, out_features=10)
    )

  def forward(self, x):
    out = self.cnn_layers(x)
    out = out.view(-1, 512*7*7)
    out = self.avgpool(out)
    out = self.linear_layers(out)
    return out

In [ ]:
import torchvision.models as models
model = models.vgg16().to(device)
# model = VggNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
n_total_steps = len(train_loader)

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()
running_loss = 0.0
running_correct = 0.0
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    #Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward pass, update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    _, predicted = torch.max(outputs.data, 1)
    running_correct += (predicted == labels).sum().item()

    if (i+1) % 10 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
      writer.add_scalar('training loss', running_loss / 100, epoch * n_total_steps + i)
      writer.add_scalar('accuracy', running_correct / 100, epoch * n_total_steps + i)
      running_loss = 0.0
      running_correct = 0.0
      FILE = 'model.pth'
      torch.save(model, FILE)

end.record()
torch.cuda.synchronize()
print('Training Done')
print(start.elapsed_time(end))


Epoch [1/30], Step [10/1250], Loss: 6.8141
Epoch [1/30], Step [20/1250], Loss: 5.2584
Epoch [1/30], Step [30/1250], Loss: 4.3395
Epoch [1/30], Step [40/1250], Loss: 3.0606
Epoch [1/30], Step [50/1250], Loss: 2.6897
Epoch [1/30], Step [60/1250], Loss: 2.3885
Epoch [1/30], Step [70/1250], Loss: 2.4557
Epoch [1/30], Step [80/1250], Loss: 2.2340
Epoch [1/30], Step [90/1250], Loss: 2.4720
Epoch [1/30], Step [100/1250], Loss: 2.4118
Epoch [1/30], Step [110/1250], Loss: 2.3453
Epoch [1/30], Step [120/1250], Loss: 2.3568
Epoch [1/30], Step [130/1250], Loss: 2.2873
Epoch [1/30], Step [140/1250], Loss: 2.4258
Epoch [1/30], Step [150/1250], Loss: 2.2238
Epoch [1/30], Step [160/1250], Loss: 2.2677
Epoch [1/30], Step [170/1250], Loss: 2.3715
Epoch [1/30], Step [180/1250], Loss: 2.2947
Epoch [1/30], Step [190/1250], Loss: 2.2372
Epoch [1/30], Step [200/1250], Loss: 2.2885
Epoch [1/30], Step [210/1250], Loss: 2.1641
Epoch [1/30], Step [220/1250], Loss: 2.2905
Epoch [1/30], Step [230/1250], Loss: 2.21

In [ ]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]
  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1

  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network: {acc}%')

  for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc}%')

Accuracy of the network: 78.53%
Accuracy of plane: 81.9%
Accuracy of car: 86.6%
Accuracy of bird: 62.0%
Accuracy of cat: 58.1%
Accuracy of deer: 73.7%
Accuracy of dog: 74.5%
Accuracy of frog: 84.7%
Accuracy of horse: 86.4%
Accuracy of ship: 88.7%
Accuracy of truck: 88.7%
